In [2]:
! pip install -U sentence-transformers huggingface_hub

   ---------------------------------------- 0.0/566.3 kB ? eta -:--:--
   ---------------------------------------- 566.3/566.3 kB 20.3 MB/s  0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0



[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import os
import re
import torch
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util

# 환경 설정 및 타임아웃 방지
os.environ['HF_HUB_READ_TIMEOUT'] = '100'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 1. 모델 로드 (한국어 문맥 파악에 최적화)
model_name = 'jhgan/ko-sroberta-multitask'
model = SentenceTransformer(model_name, device=device)

root_path = r'G:\내 드라이브\빅데이터_부산\텍스트(네이버 리뷰)' 
preprocessed_data = []

def clean_text(text):
    if pd.isna(text): return ""
    text = str(text)
    
    # 1. 제거할 문구 리스트 (더 강력하게 보강)
    noise_patterns = [
        '소정의 원고료', '재배포 금지', '무단 전재', '내돈내산', '협찬받아', 
        '부산 여행', '부산 가볼만한곳', '부산 데이트', '부산 나들이', '추천합니다'
    ]
    
    for p in noise_patterns:
        text = text.replace(p, "")
        
    # 2. 의미 없는 공백 및 특수문자 정리
    text = re.sub(r'[^가-힣0-9a-zA-Z\s]', ' ', text)
    return text.strip()

print(f"[{model_name}] 부산 관광지 감성/맥락 분석 시작... (장치: {device})")

# 2. 데이터 순회 및 임베딩 생성
for addr, dirs, files in os.walk(root_path):
    for file_name in files:
        if file_name.endswith(('.csv', '.xlsx')):
            file_path = os.path.join(addr, file_name)
            place_name = f"{os.path.basename(addr)}_{os.path.splitext(file_name)[0]}"
            
            try:
                df = pd.read_csv(file_path) if file_name.endswith('.csv') else pd.read_excel(file_path)
                
                blog_texts = []
                for _, row in df.iterrows():
                    title = clean_text(row.get('제목', ''))
                    content = clean_text(row.get('본문', ''))
                    
                    # [전략] 제목 5번 반복으로 가중치 부여 + 본문 800자 (맥락 파악용)
                    combined_text = (f"{title} " * 2) + content[:1200]
                    if len(combined_text) > 30: 
                        blog_texts.append(combined_text)

                # 최소 5개 이상의 리뷰가 있어야 신뢰도 확보
                if len(blog_texts) >= 5:
                    with torch.no_grad():
                        # 관광지별 '평균 벡터' 생성
                        embeddings = model.encode(blog_texts, batch_size=32, convert_to_tensor=True)
                        place_vector = torch.mean(embeddings, dim=0).cpu().numpy()
                    
                    preprocessed_data.append({'place_name': place_name, 'vector': place_vector})
                    print(f"  - [임베딩 완료] {place_name}")
                
            except Exception as e:
                print(f"  - [오류] {place_name}: {e}")

# 3. 유사도 점수 변별력 강화 (Zero-Centering 후처리)
print("\n변별력 강화를 위한 벡터 보정 및 유사도 계산 중...")

names = [d['place_name'] for d in preprocessed_data]
vectors = np.array([d['vector'] for d in preprocessed_data])

# [핵심] 모든 벡터에서 전체 평균을 빼서 각 장소만의 특징을 극대화함 (점수 편중 해결)
centroid = np.mean(vectors, axis=0)
centered_vectors = vectors - centroid

# 코사인 유사도 계산
sim_matrix = util.cos_sim(centered_vectors, centered_vectors).numpy()

# 4. 결과 저장 (가로 확장형)
final_rows = []
for i, base_name in enumerate(names):
    scores = sim_matrix[i]
    sorted_idx = np.argsort(scores)[::-1]
    
    row = {'기준_관광지': base_name}
    rank = 1
    for idx in sorted_idx:
        target_name = names[idx]
        if base_name != target_name:
            # 점수가 더 넓게 퍼진 상태로 저장됨
            row[f'유사_관광지_{rank}'] = f"{target_name}({scores[idx]:.4f})"
            rank += 1
            if rank > 10: break
    final_rows.append(row)

pd.DataFrame(final_rows).to_csv('busan_sbert_final_recommend.csv', index=False, encoding='utf-8-sig')
print("\n[최종 완료] 결과 파일: busan_sbert_final_recommend.csv")

[jhgan/ko-sroberta-multitask] 부산 관광지 감성/맥락 분석 시작... (장치: cpu)
  - [임베딩 완료] 남구_부산박물관
  - [임베딩 완료] 남구_용호만부두
  - [임베딩 완료] 남구_부산항대교
  - [임베딩 완료] 남구_오륙도
  - [임베딩 완료] 남구_이기대도시자연공원
  - [임베딩 완료] 남구_UN공원
  - [임베딩 완료] 해운대구_벡스코
  - [임베딩 완료] 해운대구_더베이101
  - [임베딩 완료] 해운대구_해운대블루라인파크
  - [임베딩 완료] 해운대구_해운대해수욕장
  - [임베딩 완료] 해운대구_송정해수욕장
  - [임베딩 완료] 해운대구_누리마루
  - [임베딩 완료] 해운대구_영화의전당
  - [임베딩 완료] 해운대구_장산
  - [임베딩 완료] 북구_화명생태공원
  - [임베딩 완료] 북구_문화빙상센터
  - [임베딩 완료] 북구_구포시장
  - [임베딩 완료] 북구_부산어촌민속관
  - [임베딩 완료] 부산진구_전포카페거리
  - [임베딩 완료] 부산진구_황령산
  - [임베딩 완료] 부산진구_송상현광장
  - [임베딩 완료] 부산진구_호천마을
  - [임베딩 완료] 부산진구_부전마켓타운
  - [임베딩 완료] 부산진구_성지곡수원지
  - [임베딩 완료] 동래구_금강공원
  - [임베딩 완료] 동래구_동래온천
  - [임베딩 완료] 동래구_동래읍성
  - [임베딩 완료] 중구_자갈치시장
  - [임베딩 완료] 중구_용두산공원
  - [임베딩 완료] 중구_보수동책방골목
  - [임베딩 완료] 중구_국제시장
  - [임베딩 완료] 중구_비프광장
  - [임베딩 완료] 사하구_감천문화마을
  - [임베딩 완료] 사하구_부네치아
  - [임베딩 완료] 사하구_아미산전망대
  - [임베딩 완료] 사하구_다대포해수욕장
  - [임베딩 완료] 사하구_낙동강하구에코센터
  - [임베딩 완료] 사하구_을숙도생태공원
  - [임베딩 완료] 사하구_다대포낙조분수
  - [임베딩 완료] 금정구_회동수원지
  - [

In [6]:
pd.DataFrame(final_rows).to_csv('busan_sbert_final_recommend.csv', index=False, encoding='utf-8-sig')
print("\n[최종 완료] 결과 파일: busan_sbert_final_recommend.csv")


[최종 완료] 결과 파일: busan_sbert_final_recommend.csv


In [8]:
# 5. 모든 관광지 간의 유사도 매트릭스 저장 (정방 행렬 형태)
print("\n'감상분석 유사도' 매트릭스 생성 중...")

# 유사도 매트릭스를 데이터프레임으로 변환 (인덱스와 컬럼명을 관광지명으로 설정)
df_similarity_matrix = pd.DataFrame(sim_matrix, index=names, columns=names)

# 파일 저장
output_filename = '감상분석 유사도.csv'
df_similarity_matrix.to_csv(output_filename, encoding='utf-8-sig')

print(f"[추가 완료] 모든 관광지 간 유사도 표 저장 완료: {output_filename}")


'감상분석 유사도' 매트릭스 생성 중...
[추가 완료] 모든 관광지 간 유사도 표 저장 완료: 감상분석 유사도.csv
